# StableTuner v0.1.1 for Linux

Original version https://github.com/devilismyfriend/StableTuner
<br>
STv1.0 Update (12/16/2022)
<br><br>
Testing on docker image dnwalkup/cuda:116-cudnn8-devel-u2004:
<br>
* Ubuntu 20.04, Nvidia CUDNN 8, CUDA 1.16 <br>
* PIP'd - gDown, wget, ftfy, OmegaConf, tqdm, tensorboard, transfomers, triton, pillow, iPython, pycuda, ipywidgets, jupyterlab <br>
* APT'd - zip, unzip, rename, python3, python3-apt, python3-diskutils, python3-pip <br>

### Set Global Variables and Imports

In [ ]:
# ***** DATASET ***** #
goog_dataset_url = ''
# Google url for your training dataset (e.g. instance images or user images). Use zip file.

backup_dataset_url = ''
# Hugging face, dropbox, etc. if google is capping your bandwidth for your training dataset (e.g. instance images or user images). Use zip file.

dataset_repeats = 1
# How many times would you like to repeat the dataset? The default of 1 here usually works well.


# ***** PRIOR PRESERVATION LOSS MITIGATION ***** #
use_regularization = False
# Do you want to use regularization & prior preservation loss (e.g. class images) in your training? If so, this should be True.

goog_regimg_url = ''
# Google url for your regularization images (e.g. class images). Use zip file.

backup_regimg_url = ''
# Hugging face, dropbox, etc. if google is capping your bandwith for your regularization images (e.g. class images). Use zip file.


# ***** GENERAL SETTINGS ***** #
sd_base_model = 'SDv15'
# Input the model you'd like to use. Options are 'SDv14', 'SDv15', 'SDv20-512', 'SDv20-768', 'SDv21-512', 'SDv21-768'. Default is 'SDv15'.

seed = ''
# Leave empty for a random seed.

google_data_cap = False
# Is google capping your bandwidth and not letting you download your files? If so, this needs to be true.

txt_file_caption = False
# If you have text files you'd like to use for captions, change to True.

aspect_ratio_bucketing = False
# If you don't want to limit yourself to square 1:1 dataset images, change to True.

sample_at_start = False
# Samples (and saves?) an image on the start of training to help track progress. Default is False.

auto_balance = False
# Will balance the number of images in each concept dataset to match the minimum number of images in any concept dataset

save_n_epoch = 5
# Save every n epochs to help prevent overtraining.

train_txt_encoder = True
# Whether or not to train the text encoder.

mixed_precision = 'fp16'
# Whether to use mixed precision. Choose between 'no', 'fp16' and 'bf16'.
# Bf16 requires PyTorch >= 1.10 and an Nvidia Ampere GPU.


# ***** ADVANCED SETTINGS ***** #
prior_preservation_weight = 1.0
# Variable only used if 'use_class_images' is True.

cudnn_benchmark = False
# This enables or disables CUDNN benchmarking. Default is disabled (False).

txt_encoder_training_epoch = 999999999999999
# The epoch at which the text encoder is no longer trained

new_latent_cache = True
# Regenerate latent cache. Necessary if you've made changes to batch size.

save_latents_cache = False
# Needs description

add_reg_img_dataset = False
# Will generate and/or add existing regularization images to the dataset (does not enable prior preservation loss).

train_1024 = False
# If you want to train at 1024 resolution, change to true.


import os, gc, random
from subprocess import getoutput
from IPython.display import clear_output

clear_output()
print('Imports complete')

### Install Apt Dependencies for Linux

In [ ]:
!apt update
!apt install -y git
!apt install -y nvidia-cuda-toolkit
!apt install -y zip
!apt install -y unzip

clear_output()
print('Apt installs complete')

### Install Pip Dependencies

In [ ]:
!python3 -m pip install scikit-image
!python3 -m pip install scipy
!python3 -m pip install numpy
!python3 -m pip install requests
!python3 -m pip install bitsandbytes
!python3 -m pip install accelerate==0.15.0
!python3 -m pip install huggingface
!python3 -m pip install albumentations
!python3 -m pip install opencv-python
!python3 -m pip install einops
!python3 -m pip install pytorch_lightning
!python3 -m pip install bitsandbytes==0.35.0
!python3 -m pip install safetensors
!python3 -m pip install diffusers

clear_output()
print('Pip installs complete')

### Install xFormers and Final Dependencies

In [ ]:
GPU_CardName = getoutput('nvidia-smi --query-gpu=name --format=csv,noheader')

if '3090' in GPU_CardName:
    !python3 -m pip install https://huggingface.co/dnwalkup/xformers-precompiles/resolve/main/RTX3090-xf14-cu116-py38/xformers-0.0.14.dev0-cp38-cp38-linux_x86_64.whl
    !python3 -m pip uninstall -y torch torchvision
    !python3 -m pip install https://download.pytorch.org/whl/cu116/torch-1.12.1%2Bcu116-cp38-cp38-linux_x86_64.whl
    !python3 -m pip install https://download.pytorch.org/whl/cu116/torchvision-0.13.1%2Bcu116-cp38-cp38-linux_x86_64.whl
    !python3 -m pip install timm
    clear_output()
    print('xFormers and pyTorch installs complete')
elif 'A5000' in GPU_CardName:
    !python3 -m pip install https://huggingface.co/dnwalkup/xformers-precompiles/resolve/main/A5000-xf14-cu116-py38/xformers-0.0.14.dev0-cp38-cp38-linux_x86_64.whl
    !python3 -m pip uninstall -y torch torchvision
    !python3 -m pip install https://download.pytorch.org/whl/cu116/torch-1.12.1%2Bcu116-cp38-cp38-linux_x86_64.whl
    !python3 -m pip install https://download.pytorch.org/whl/cu116/torchvision-0.13.1%2Bcu116-cp38-cp38-linux_x86_64.whl
    !python3 -m pip install timm
    clear_output()
    print('xFormers and pyTorch installs complete')
else:
    print('No prebuilt xformers available for your card')

del GPU_CardName
gc.collect()

### Download Necessary Files

In [ ]:
os.mkdir('/workspace/scripts')
os.chdir('/workspace/scripts')

os.system('wget -O trainer.py https://github.com/dnwalkup/StableTuner/raw/main/linux/scripts/trainer.py')
os.system('wget -O converters.py https://github.com/dnwalkup/StableTuner/raw/main/linux/scripts/converters.py')

clear_output()
print('Training files downloaded')

### Download Training Dataset

In [ ]:
os.mkdir('/workspace/dataset')
os.chdir('/workspace/dataset')

if not google_data_cap:
    os.system('gdown --fuzzy $goog_dataset_url -O dataset.zip')
else:
    os.system('wget -O dataset.zip $other_dataset_url')

os.system('unzip dataset.zip')
os.remove('/workspace/dataset/dataset.zip')

for instance_items in os.scandir(os.getcwd()):
    if instance_items.is_dir():
        INSTANCE_DIR_TMP = instance_items.path

os.rename(INSTANCE_DIR_TMP,"user_images")

DATASET_DIR = '/workspace/dataset/user_images'

os.chdir(DATASET_DIR)
os.system('find . -name "* *" -type f | rename "s/ /_/g"')

del instance_items
del INSTANCE_DIR_TMP
gc.collect()

clear_output()
print('Instance images downloaded and unzipped')

### Download Regularization Images
Only used if you have "use_regularization" set to True

In [ ]:
if use_regularization:

    os.mkdir('/workspace/regularization')
    os.chdir('/workspace/regularization')

    if not google_data_cap:
        os.system('gdown --fuzzy $goog_regimg_url -O reg_images.zip')
    else:
        os.system('wget -O reg_images.zip $other_regimg_url')

    os.system('unzip reg_images.zip')
    os.remove('reg_images.zip')

    for reg_items in os.scandir(os.getcwd()):
        if reg_items.is_dir():
            REG_DIR_TMP = reg_items.path

    os.rename(REG_DIR_TMP,'reg_images')

    CLASS_DIR = '/workspace/regularization/reg_images'

    os.chdir(CLASS_DIR)
    os.system('find . -name "* *" -type f | rename "s/ /_/g"')

    del reg_items
    del REG_DIR_TMP
    gc.collect()

    clear_output()
    print('Regularization images downloaded and unzipped')
else:
    print('Please move on to the next cell')

### Set Final Training Variables

In [ ]:
if cudnn_benchmark:
    cudnn_benchmark_args = ''
else:
    cudnn_benchmark_args = '--disable_cudnn_benchmark=True'

if txt_file_caption:
    txt_file_caption_args = '--use_text_files_as_captions=True'
else:
    txt_file_caption_args = ''

if aspect_ratio_bucketing:
    aspect_ratio_bucketing_args = '--use_bucketing=True'
    sample_aspect_ratio_args = '--sample_aspect_ratios'
else:
    aspect_ratio_bucketing_args = ''
    sample_aspect_ratio_args = ''

if new_latent_cache:
    new_latent_cache_args = '--regenerate_latent_cache=True'
else:
    new_latent_cache_args = ''

if save_latents_cache:
    save_latents_cache_args = '--save_latents_cache=True'
else:
    save_latents_cache = ''

if sample_at_start:
    sample_at_start_args = '--sample_on_training_start=True'
else:
    sample_at_start_args = ''

if add_reg_img_dataset:
    add_reg_img_dataset_args = '--add_class_images_to_dataset=True'
else:
    add_reg_img_dataset_args = ''

if auto_balance:
    auto_balance_args = '--auto_balance_concept_datasets=True'
else:
    auto_balance_args = ''

if train_txt_encoder:
    train_txt_encoder_args = '--train_text_encoder=True'
else:
    train_txt_encoder = ''

if sd_base_model == 'SDv14':
    sd_base_model_args = '--pretrained_model_name_or_path=CompVis/stable-diffusion-v1-4'
    resolution_args = '--resolution=512'
elif sd_base_model == 'SDv20-512':
    sd_base_model_args = '--pretrained_model_name_or_path=stabilityai/stable-diffusion-2-base'
    resolution_args = '--resolution=512'
elif sd_base_model == 'SDv20-768':
    sd_base_model_args = '--pretrained_model_name_or_path=stabilityai/stable-diffusion-2'
    resolution_args = '--resolution=768'
elif sd_base_model == 'SDv21-512':
    sd_base_model_args = '--pretrained_model_name_or_path=stabilityai/stable-diffusion-2-1-base'
    resolution_args = '--resolution=512'
elif sd_base_model == 'SDv21-768':
    sd_base_model_args = '--pretrained_model_name_or_path=stabilityai/stable-diffusion-2-1'
    resolution_args = '--resolution=768'
else:
    sd_base_model_args = '--pretrained_model_name_or_path=runwayml/stable-diffusion-v1-5'
    resolution_args = '--resolution=512'

if train_1024:
    resolution_args = '--resolution=1024'

if use_regularization:
    prior_preservation = ''
    #prior_preservation_weight is user set
    class_dir_args = '--class arg=$CLASS_DIR'
else:
    prior_preservation = ''
    class_dir_args = ''

if seed == '' or seed == '0':
  seed = random.randint(1, 999999)
else:
  seed = int(seed)

output_dir = '/workspace/output_model'


### Launch Training

In [ ]:
!accelerate launch --mixed_precision=$mixed_precision /workspace/scripts/trainer.py \
    $cudnn_benchmark_args \
    $txt_file_caption_args \
    $sd_base_model_args \
    "--pretrained_vae_name_or_path=" \
    --output_dir=$output_dir \
    --seed=$seed \
    $resolution_args \
    "--train_batch_size=1" \
    "--num_train_epochs=100" \
    --mixed_precision=$mixed_precision \
    --stop_text_encoder_training=$txt_encoder_training_epoch \
    $aspect_ratio_bucketing_args \
    $sample_aspect_ratio_args \
    "--use_8bit_adam" \
    "--gradient_checkpointing" \
    "--gradient_accumulation_steps=1" \
    "--learning_rate=3e-6" \
    "--lr_warmup_steps=0" \
    "--lr_scheduler=constant" \
    $new_latent_cache_args \
    $save_latents_cache_args \
    "--concepts_list=/workspace/stabletune_concept_list.json" \
    $auto_balance_args
    "--num_class_images=0" \
    $add_reg_img_dataset_args \
    --save_every_n_epoch=$save_n_epoch \
    $train_txt_encoder \
    $sample_at_start_args \
    "--sample_height=512" \
    "--sample_width=512" \
    --dataset_repeats=$dataset_repeats